In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of USFWS biologist labels of imagery from Bosque del Apache and Maxwell NWR

In [1]:
#Imports
import pandas as pd

In [2]:
#Analysis annotations
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_expertanalysislabels.csv"
with open(path) as f:
  df = pd.read_csv(f)

#Refined annotations
refinedpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_expertconsensuslabels.csv"
with open(refinedpath) as f:
  ref = pd.read_csv(f)

In [4]:
df.head()

,Unnamed: 0,annotation_ID,bbox_orig,filename,labeler,cat_orig,cluster_id,cat_refined,bbox_refined
0,0,1,"[4428, 2707, 125, 103]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,0,Canadian Goose,"[4445.5, 2719.5, 95.0, 80.5]"
1,1,2,"[4308, 2731, 105, 67]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,1,Canadian Goose,"[4312.5, 2739.5, 98.0, 44.0]"
2,2,3,"[3707, 1761, 110, 101]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,2,Canadian Goose,"[3725.5, 1779.0, 73.5, 70.5]"
3,3,4,"[3628, 1882, 90, 38]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,3,Canadian Goose,"[3628.0, 1882.0, 92.0, 38.0]"
4,4,5,"[3669, 1927, 69, 82]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,4,Canadian Goose,"[3679.0, 1929.0, 65.0, 82.0]"


In [10]:
clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()
c = clusters.reset_index()
c["expected"] = c["bbox_orig"]/c["labeler"]
c["diff"] = c["cluster_id"]/c["expected"]
clusters
#pd.eval("c['diff'] > 1.1 or c['diff'] < 0.9").sum()

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_21390/3555870435.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()


,bbox_orig,cluster_id,labeler
filename,,,
BDA_12C_20181127_1.JPG,841,86,10
BDA_12C_20181127_2.JPG,3817,489,8
BDA_12C_20181127_3.JPG,4867,793,6
BDA_18A4_20181106_1.JPG,1622,164,10
BDA_18A4_20181106_2.JPG,3497,356,10
BDA_18A4_20181106_3.JPG,438,63,7
BDA_18A4_20181106_4.JPG,1145,116,10
BDA_18A4_20181107_1.JPG,780,79,10
BDA_18A4_20181107_2.JPG,201,27,7


In [6]:
#Refined annotation statistics
print(len(ref))
print(ref["category"].value_counts())

2375
Mallard                                                               1688
Northern Pintail                                                       262
Canadian Goose                                                         140
Other                                                                   70
Sandhill Crane                                                          52
['Mallard' 'Other']                                                     33
American Wigeon                                                         22
['Mallard' 'Northern Pintail']                                          20
['Northern Pintail' 'Other']                                            15
['American Wigeon' 'Mallard']                                           15
['American Wigeon' 'Mallard' 'Northern Pintail' 'Other' 'Teal']          9
['American Wigeon' 'Other']                                              7
Gadwall                                                                  5
['Gadwall' 'Mallard'

In [ ]:
#INDIVIDUAL LABELER METRICS
#Goal: report avg count, ID consensus across images, break down by morphological group; also min/max individual vs group consensus
#Positional error: create confusion matrix, report commission error, omission error, RMSE, MAE -- bounding boxes, comparing each labeler to the consensus

#Score agreement with consensus class label

#Score %IOU agreement with consensus box 

In [ ]:
#OUTPUTS

#Line graph of average identification consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Bar graph of consensus (y; % agreement) by morphology (x; duck/goose/crane). Two bars per class, one for expert, one for crowdsourced
#Line graph of average count consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Line graph of consensus (y; % agreement) by density (x; consensus # of individuals per image). One line for expert, one line for crowdsourced.
